<a href="https://colab.research.google.com/github/raznuss/global_urban_flood_database/blob/main/urban_flood_visualiztion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== Colab: Display one flood event by event_id (GHSL-aligned, centroid-cluster as polygon) =====
# Black = urban area of the selected city cluster
# Red   = urban flood within that city cluster
# Blue  = total flood excluding permanent water

import ee, geemap
import pandas as pd
import numpy as np

# ===================== 1) User params =====================
PROJECT_ID = "iconic-exchange-477008-h2"

# CSV upload is used to pick the row by event_id and read gfd_event_id, ghsl_year_used, centroid, threshold
USE_UPLOAD = True
FIXED_CSV_PATH = "/content/urban_flood_database.csv"

EVENT_ID = 1470                   # <-- set the event_id you want to display
URBAN_THRESHOLD_DEFAULT = 0.20    # will be overridden by CSV if present

# Window around centroid to isolate the relevant city cluster
ROI_KM        = 80

# Resolutions similar or higher than the original generator
AREA_SCALE_M  = 120    # for fast stats
POLY_SCALE_M  = 100    # vectorization scale for polygons (GHSL native is ~100 m)
TILE_SCALE    = 12

# Visualization styles
STYLE_URBAN  = {'palette': ['#000000'], 'min': 0, 'max': 1, 'opacity': 0.40}
STYLE_UF     = {'palette': ['#d73027'], 'min': 0, 'max': 1, 'opacity': 0.65}
STYLE_FLOOD  = {'palette': ['#2b8cbe'], 'min': 0, 'max': 1, 'opacity': 0.45}

# ===================== 2) EE init =====================
def init_ee(project_id: str):
    try:
        ee.Initialize(project=project_id)
    except Exception:
        ee.Authenticate()
        ee.Initialize(project=project_id)
    print("EE initialized:", project_id)

# ===================== 3) Helpers =====================
GFD = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

def safe_int(x):
    try:
        if pd.isna(x) or str(x).strip() == "":
            return None
        return int(float(str(x)))
    except Exception:
        return None

def load_csv_row():
    if USE_UPLOAD:
        from google.colab import files
        print("Please upload your latest urban_flood_database.csv from your PC…")
        uploaded = files.upload()
        if not uploaded:
            raise RuntimeError("No file uploaded.")
        csv_path = list(uploaded.keys())[0]
    else:
        csv_path = FIXED_CSV_PATH

    print("Using CSV:", csv_path)
    db = pd.read_csv(csv_path, low_memory=False)
    print("\nFirst 20 event_id values in the CSV:")
    try:
        print(pd.to_numeric(db['event_id'], errors='coerce').head(20).tolist())
    except Exception:
        pass
    row = db.loc[pd.to_numeric(db['event_id'], errors='coerce') == EVENT_ID]
    if row.empty:
        raise ValueError(f"event_id={EVENT_ID} not found in CSV")
    return row.iloc[0]

def ghsl_image_for_year(y: int) -> ee.Image:
    valid = list(range(1975, 2035, 5))  # 1975, 1980, ..., 2030
    if y not in valid:
        y = min(valid, key=lambda e: abs(e - y))
    return ee.Image(f'JRC/GHSL/P2023A/GHS_BUILT_S/{int(y)}')

def ghsl_with_fallback(year_preferred: int, geom: ee.Geometry):
    """
    Try the requested GHSL year. If built_surface has no signal within geom,
    fall back to nearby epochs. Return (image, year_used, stats_dict, tried_list).
    """
    candidates = [year_preferred, 2020, 2015, 2010, 2000, 1990, 1975, 2030]
    tried = []
    for y in candidates:
        img  = ghsl_image_for_year(int(y))
        band = img.select('built_surface').clip(geom)
        stats = ee.Dictionary({
            'min':  band.reduceRegion(ee.Reducer.min(),  geom, 100, 1e13, TILE_SCALE, True).get('built_surface'),
            'max':  band.reduceRegion(ee.Reducer.max(),  geom, 100, 1e13, TILE_SCALE, True).get('built_surface'),
            'mean': band.reduceRegion(ee.Reducer.mean(), geom, 100, 1e13, TILE_SCALE, True).get('built_surface'),
        }).getInfo() or {}
        tried.append((y, stats))
        mx = float(stats.get('max') or 0)
        if mx > 0:
            return img, int(y), stats, tried
    # fallback to preferred even if empty
    return ghsl_image_for_year(int(year_preferred)), int(year_preferred), (tried[-1][1] if tried else {}), tried

def load_gfd_image(gfd_id: int) -> ee.Image:
    img = ee.Image(GFD.filter(ee.Filter.eq('id', gfd_id)).first())
    if img.getInfo() is None:
        img = ee.Image(GFD.filter(ee.Filter.eq('system:index', str(gfd_id))).first())
        if img.getInfo() is None:
            raise ValueError(f"GFD image not found for id={gfd_id}")
    return img

def build_roi_from_centroid(centroid_text: str, fallback_geom: ee.Geometry):
    if not isinstance(centroid_text, str) or not centroid_text.strip():
        return fallback_geom
    try:
        lon_str, lat_str = centroid_text.split(',')
        pt = ee.Geometry.Point([float(lon_str.strip()), float(lat_str.strip())])
        return pt.buffer(ROI_KM * 1000).bounds(1000)
    except Exception:
        return fallback_geom

def pick_city_polygon(urban_mask_roi: ee.Image, centroid_text: str, roi: ee.Geometry) -> ee.Geometry:
    """
    Vectorize the urban mask within ROI and pick the polygon that contains the centroid.
    If centroid is missing or not inside any polygon, pick the largest polygon within ROI.
    """
    polys = urban_mask_roi.selfMask().reduceToVectors(
        geometry=roi,
        scale=POLY_SCALE_M,
        geometryType='polygon',
        labelProperty='urban',
        maxPixels=1e13,
        tileScale=TILE_SCALE,
        bestEffort=True
    )

    # Try centroid containment first
    if isinstance(centroid_text, str) and centroid_text.strip():
        try:
            lon_str, lat_str = centroid_text.split(',')
            pt = ee.Geometry.Point([float(lon_str.strip()), float(lat_str.strip())])
            polys_with_pt = polys.filterBounds(pt)
            polys_area    = polys_with_pt.map(lambda f: f.set('area', f.geometry().area(1))).sort('area', False)
            best          = polys_area.first()
            best = ee.Feature(ee.Algorithms.If(
                best,
                best,
                polys.map(lambda f: f.set('area', f.geometry().area(1))).sort('area', False).first()
            ))
            if best.getInfo() is not None:
                return ee.Feature(best).geometry()
        except Exception as e:
            print("Centroid parse failed:", e)

    # Fallback: largest polygon in ROI
    polys_area = polys.map(lambda f: f.set('area', f.geometry().area(1))).sort('area', False)
    biggest = polys_area.first()
    if biggest.getInfo() is not None:
        return ee.Feature(biggest).geometry()
    return roi

def quick_sum(img: ee.Image, band: str, geom: ee.Geometry) -> float:
    v = img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=AREA_SCALE_M,
        maxPixels=1e13,
        tileScale=TILE_SCALE,
        bestEffort=True
    ).get(band)
    try:
        return float(ee.Number(v).getInfo() or 0)
    except Exception:
        return 0.0

# ===================== 4) Core compute =====================
def compute_layers_for_event(row: pd.Series):
    # Parse row
    gfd_id     = safe_int(row.get('gfd_event_id'))
    ghsl_year0 = safe_int(row.get('ghsl_year_used')) or 2020
    centroid   = row.get('cluster_centroid_xy_round', None)
    thr_raw    = row.get('urban_threshold_frac')
    urban_thr  = float(thr_raw) if pd.notna(thr_raw) else URBAN_THRESHOLD_DEFAULT

    if gfd_id is None:
        raise ValueError("Missing gfd_event_id in CSV for the selected row")

    # Load the event image
    gfd_img = load_gfd_image(gfd_id)
    geom_event = gfd_img.geometry()
    roi  = build_roi_from_centroid(centroid, geom_event)
    geom = ee.Geometry(geom_event.intersection(roi, ee.ErrorMargin(1000)))

    # GHSL with fallback if the requested year has no signal inside the event geometry
    ghsl_img, ghsl_year_used, ghsl_stats, ghsl_tried = ghsl_with_fallback(ghsl_year0, geom)
    print(f"GHSL year used: {ghsl_year_used}")
    print("GHSL stats in event geom:", ghsl_stats)
    if ghsl_tried and len(ghsl_tried) > 1:
        print("GHSL fallbacks tried (year, max):", [(y, (s or {}).get('max', None)) for y, s in ghsl_tried])

    ghsl_proj = ghsl_img.select('built_surface').projection()

    # Flood layers then align to GHSL projection
    flooded = gfd_img.select('flooded').eq(1).unmask(0).clip(geom)
    perm    = gfd_img.select('jrc_perm_water').eq(1).unmask(0).clip(geom)
    fnp     = flooded.And(perm.Not()).rename('fnp').unmask(0).clip(geom).reproject(ghsl_proj)

    # Urban mask from GHSL
    built_m2   = ghsl_img.select('built_surface').clip(geom)
    built_frac = built_m2.divide(10000.0)
    urban_mask = built_frac.gte(urban_thr).rename('urban').unmask(0).clip(geom).reproject(ghsl_proj)

    # Limit to ROI and pick the city polygon by centroid or largest
    urban_mask_roi = urban_mask.updateMask(urban_mask).clip(roi)
    city_poly = pick_city_polygon(urban_mask_roi, centroid, roi)

    # Rasterize the chosen polygon into the GHSL projection
    city_mask = (ee.Image().paint(city_poly, 1, 0)
                 .rename('urban').unmask(0).toByte().reproject(ghsl_proj))

    # Keep only truly urban pixels in that polygon
    city_mask = city_mask.And(urban_mask).rename('urban').unmask(0)

    # Urban flood inside city
    uf_city = fnp.And(city_mask).rename('uf').unmask(0)

    return {
        "geom": geom,
        "city_mask": city_mask,
        "urban_flood": uf_city,
        "flood_np": fnp
    }

# ===================== 5) Map =====================
def make_map(city_mask, uf_city, flood_np, geom):
    s_urban = quick_sum(city_mask, 'urban', geom)
    s_uf    = quick_sum(uf_city,  'uf',    geom)
    s_fnp   = quick_sum(flood_np, 'fnp',   geom)

    print(f"Has urban pixels: {s_urban > 0}")
    print(f"Has urban flood: {s_uf > 0}")
    print(f"Has total flood: {s_fnp > 0}")

    m = geemap.Map(center=[0, 0], zoom=2)
    m.addLayer(flood_np.selfMask(), STYLE_FLOOD, 'Total flood non permanent')
    m.addLayer(city_mask.selfMask(), STYLE_URBAN, 'Urban area city cluster')
    m.addLayer(uf_city.selfMask(), STYLE_UF, 'Urban flood city cluster')

    # Smart centering preference: UF -> urban -> flood -> geom
    try:
        if s_uf > 0:
            m.centerObject(uf_city.selfMask(), 11)
        elif s_urban > 0:
            m.centerObject(city_mask.selfMask(), 10)
        elif s_fnp > 0:
            m.centerObject(flood_np.selfMask(), 8)
        else:
            m.centerObject(geom, 7)
    except Exception as e:
        print("Centering fallback:", e)
        m.centerObject(geom, 7)
    return m

# ===================== 6) Main =====================
def main():
    init_ee(PROJECT_ID)

    row = load_csv_row()
    print(f"\nSelected event_id: {EVENT_ID}")
    print(f"GFD id: {row.get('gfd_event_id')}, GHSL requested year: {row.get('ghsl_year_used')}, urban threshold: {row.get('urban_threshold_frac')}")
    print("Centroid:", row.get('cluster_centroid_xy_round'))

    layers = compute_layers_for_event(row)
    m = make_map(layers["city_mask"], layers["urban_flood"], layers["flood_np"], layers["geom"])
    return m

m = main()
m


EE initialized: iconic-exchange-477008-h2
Please upload your latest urban_flood_database.csv from your PC…


Saving urban_flood_database.csv to urban_flood_database (17).csv
Using CSV: urban_flood_database (17).csv

First 20 event_id values in the CSV:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Selected event_id: 1470
GFD id: 2900, GHSL requested year: 2005, urban threshold: 0.2
Centroid: 106.4694,29.5252


EEException: Invalid argument specified for ee.List(): 12

In [ ]:
GEMINI

In [ ]:
# ===== Colab notebook: plot one event with 3 layers (FIX 3) =====
# 1. Total flood (blue)
# 2. Specific urban cluster (black)
# 3. Specific urban flood (red)

import ee, geemap, math, time, gc, os
import pandas as pd
import numpy as np

# ===================== 1) EE init =====================
PROJECT_ID = "iconic-exchange-477008-h2" # או כל פרויקט EE שיש לך גישה אליו
try:
    ee.Initialize(project=PROJECT_ID)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
print("EE initialized for project:", PROJECT_ID)

# ===================== 2) Upload your latest CSV from the PC =====================
from google.colab import files
print("Please upload your 'urban_flood_database.csv' from your PC…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
CSV_PATH = list(uploaded.keys())[0]
print(f"Using uploaded CSV: {CSV_PATH}")

# ===================== 3) Choose event to plot =====================
EVENT_ID = 1470  # <-- change this to the DB event_id you want

# Map styling as requested
STYLE_FNP     = {'palette': ['#0000FF'], 'min': 0, 'max': 1, 'opacity': 0.50} # Total flood (blue)
STYLE_URBAN   = {'palette': ['#000000'], 'min': 0, 'max': 1, 'opacity': 0.65} # Urban cluster (black)
STYLE_UF      = {'palette': ['#d73027'], 'min': 0, 'max': 1, 'opacity': 0.80} # Urban flood (red)
STYLE_OUTLINE = {'color': 'black', 'width': 1}

# Processing params aligned with your generator
AREA_SCALE_M    = 500
LABEL_SCALE_M   = 2000
TILE_SCALE      = 12

# ===================== 4) Load DB and pick the row =====================
db = pd.read_csv(CSV_PATH, low_memory=False)

print("\nFirst 20 event_id values in your uploaded DB:")
print(db['event_id'].head(20).tolist())

row = db.loc[db['event_id'] == EVENT_ID]
if row.empty:
    row = db.loc[pd.to_numeric(db['event_id'], errors='coerce') == EVENT_ID]
if row.empty:
    raise ValueError(f"event_id={EVENT_ID} not found in the uploaded DB.")

row = row.iloc[0]

def _safe_int(x):
    return int(x) if pd.notna(x) and str(x).strip() != "" else None

gfd_id               = _safe_int(row.get('gfd_event_id'))
ghsl_year_used       = _safe_int(row.get('ghsl_year_used')) or 2020
urban_threshold_frac = row.get('urban_threshold_frac', 0.20) # Use from CSV or default
centroid_str         = row.get('cluster_centroid_xy_round', None)

if gfd_id is None:
    raise ValueError("gfd_event_id is missing for the selected event in your uploaded file.")

print(f"\nSelected event_id (DB): {EVENT_ID}")
print(f"GFD id: {gfd_id}, GHSL year: {ghsl_year_used}, Urban threshold: {urban_threshold_frac}")
print("Centroid present." if centroid_str and isinstance(centroid_str, str) else "No centroid → will show whole-event urban + urban-flood.")

# ===================== 5) GEE datasets and helpers =====================
GFD = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

def ghsl_image_for_year(y):
    # Load GHSL image directly by path (epochs every 5 years)
    valid_epochs = list(range(1975, 2035, 5))
    if y not in valid_epochs:
        # snap to nearest epoch
        y = min(valid_epochs, key=lambda e: abs(e - y))
    return ee.Image(f'JRC/GHSL/P2023A/GHS_BUILT_S/{y}')

# ===== ROBUST IMAGE FETCH LOGIC =====
# Fetch the specific GFD image by DFO id (image property 'id')
gfd_collection = GFD.filter(ee.Filter.eq('id', gfd_id))

# Check size using .getInfo()
size_by_id = gfd_collection.size().getInfo()

if size_by_id > 0:
    print(f"GFD image found by id={gfd_id}")
    gfd_img = gfd_collection.first()
else:
    # Not found by 'id', try by 'system:index'
    print(f"GFD image not found for id={gfd_id}. Trying 'system:index'...")
    gfd_collection = GFD.filter(ee.Filter.eq('system:index', str(gfd_id)))

    size_by_index = gfd_collection.size().getInfo()

    if size_by_index > 0:
        print(f"GFD image found by system:index={gfd_id}")
        gfd_img = gfd_collection.first()
    else:
        # Found by neither
        raise ValueError(f"GFD image not found for id={gfd_id} (checked 'id' and 'system:index').")
# ====================================

geom = gfd_img.geometry()
print("GFD footprint bounds:", geom.bounds().getInfo())

# Build GHSL and masks
ghsl = ghsl_image_for_year(ghsl_year_used)
built_frac = ghsl.select('built_surface').divide(10000.0).clip(geom)  # m^2 per 100m cell → fraction of 1ha cell
urban_mask = built_frac.gte(urban_threshold_frac).rename('urban').unmask(0).clip(geom) # All urban in event geom

flooded = gfd_img.select('flooded').eq(1).unmask(0).clip(geom)
perm    = gfd_img.select('jrc_perm_water').eq(1).unmask(0).clip(geom)
fnp     = flooded.And(perm.Not()).rename('fnp').unmask(0).clip(geom) # <-- This is the "Total Flood" layer
urban_flood = fnp.And(urban_mask).rename('uf').unmask(0).clip(geom) # All urban flood in event geom

# Connected components over urban_mask (use the band's native projection)
proj_base = ghsl.select('built_surface').projection()
labels = (urban_mask
          .reproject(proj_base.atScale(LABEL_SCALE_M))
          .selfMask()
          .connectedComponents(ee.Kernel.plus(1), maxSize=1024)
          .select('labels').toInt64().rename('label'))

# Pick the city cluster by centroid if available
def _pick_cluster_by_centroid(centroid_text):
    if not centroid_text or not isinstance(centroid_text, str):
        print("No centroid, using full urban_mask")
        return ee.Dictionary({'city_mask': urban_mask, 'uf_city': urban_flood})
    try:
        lon_str, lat_str = centroid_text.split(',')
        lon = float(lon_str.strip())
        lat = float(lat_str.strip())
    except Exception:
        print("Centroid parse error, using full urban_mask")
        return ee.Dictionary({'city_mask': urban_mask, 'uf_city': urban_flood})

    pt = ee.Geometry.Point([lon, lat])

    lab_dict = labels.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=pt,
        scale=LABEL_SCALE_M,
        maxPixels=1e8,
        bestEffort=True
    )
    lab_val = lab_dict.get('label')

    def _with_target(lbl):
        target = labels.eq(ee.Number(lbl))
        return ee.Dictionary({
            'city_mask': urban_mask.updateMask(target),
            'uf_city'  : urban_flood.updateMask(target)
        })

    # Handle cases where the point might not hit a label
    return ee.Dictionary(
        ee.Algorithms.If(
            ee.Algorithms.IsEqual(lab_val, None),
            ee.Dictionary({'city_mask': urban_mask, 'uf_city': urban_flood}), # Fallback
            _with_target(lab_val)
        )
    )

city_dict = _pick_cluster_by_centroid(centroid_str)
city_mask = ee.Image(city_dict.get('city_mask')) # <-- This is the "Specific Urban Cluster"
uf_city   = ee.Image(city_dict.get('uf_city')) # <-- This is the "Urban Flood in Cluster"

# Quick indicators for centering
uf_pixels_count = uf_city.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geom,
    scale=AREA_SCALE_M,
    maxPixels=1e13,
    tileScale=TILE_SCALE
).get('uf')

urban_pixels_count = city_mask.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=geom,
    scale=AREA_SCALE_M,
    maxPixels=1e13,
    tileScale=TILE_SCALE
).get('urban')

print("Has specific urban-flood pixels?", ee.Number(uf_pixels_count).gt(0).getInfo())
print("Has specific urban pixels?", ee.Number(urban_pixels_count).gt(0).getInfo())

# ===================== 6) Display map and center smartly =====================
m = geemap.Map(center=[0, 0], zoom=2)

# Add layers as requested: Blue -> Black -> Red
m.addLayer(fnp.selfMask(),         STYLE_FNP,    'Total Flood (non-perm)')
m.addLayer(city_mask.selfMask(),   STYLE_URBAN,  'Urban Cluster (Specific)')
m.addLayer(uf_city.selfMask(),     STYLE_UF,     'Urban Flood (in Cluster)')

# Add outline for the specific city cluster
try:
    city_outline = city_mask.selfMask().reduceToVectors(
        geometry=geom, scale=AREA_SCALE_M, geometryType='polygon',
        labelProperty='city', maxPixels=1e13, tileScale=TILE_SCALE
    )
    m.addLayer(city_outline.style(**STYLE_OUTLINE), {}, 'City Outline')
except Exception as e:
    print("City outline vectorization skipped:", e)
    city_outline = None

# Center preference: uf_city → city_mask → geom
center_zoom = 10
did_center = False
try:
    # 1) center on UF raster geometry if it has any pixels
    if ee.Number(uf_pixels_count).gt(0).getInfo():
        m.centerObject(uf_city.selfMask(), center_zoom)
        did_center = True
except Exception as e:
    print("UF center failed:", e)

if not did_center:
    try:
        if ee.Number(urban_pixels_count).gt(0).getInfo():
            m.centerObject(city_mask.selfMask(), center_zoom)
            did_center = True
    except Exception as e:
        print("City center failed:", e)

if not did_center:
    try:
        m.centerObject(geom, 7)
        did_center = True
    except Exception as e:
        print("Geom center failed:", e)

if not did_center:
    print("Falling back to default [0,0] center due to empty geometry chain.")

# Add layer control
m.addLayerControl()

# Display the map
m

EE initialized for project: iconic-exchange-477008-h2
Please upload your 'urban_flood_database.csv' from your PC…


Saving urban_flood_database.csv to urban_flood_database (14).csv
Using uploaded CSV: urban_flood_database (14).csv

First 20 event_id values in your uploaded DB:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Selected event_id (DB): 1470
GFD id: 2900, GHSL year: 2005, Urban threshold: 0.2
Centroid present.
GFD image found by id=2900
GFD footprint bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[90.53954178474983, 24.458336068959923], [113.14671158731011, 24.458336068959923], [113.14671158731011, 35.9568542804107], [90.53954178474983, 35.9568542804107], [90.53954178474983, 24.458336068959923]]]}
Has specific urban-flood pixels? 0
Has specific urban pixels? 0


Map(center=[30.254592559005108, 101.8430597263466], controls=(WidgetControl(options=['position', 'transparent_…